# About
This experiment was designed as the main focus of my extended essay for IBO. It compares the practical execution times of the AlphaTensor algorithms, the Naive method, and Strassen's algorithm for matrix multiplication.

The results folder contains execution times of the experiment ran on an M2 MacBook Air 16GB. The Python version used is 3.10.15.

# Setup

## Import Dependencies

In [ ]:
import numpy as np
import timeit
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
from utils import _generate_random_matrices, block_split, algorithm_from_factors, generate_naive_factorization, _get_n_from_factors, _get_2x2x2_strassen, pad, _generate_random_int_matrices

## Load Files

In [ ]:
standard_factorizations_path = "/Users/kev/Documents/projects/alphatensor-analysis/algorithms/factorizations_r.npz"
mod2_factorizations_path = "/Users/kev/Documents/projects/alphatensor-analysis/algorithms/factorizations_f2.npz"

standard_factorizations = dict(np.load(standard_factorizations_path, allow_pickle=True))
mod2_factorizations = dict(np.load(mod2_factorizations_path, allow_pickle=True))

## Print Ranks

In [ ]:
# Print available factorizations and their shapes.
for key in standard_factorizations:
  naive_u, v, w = standard_factorizations[key]
  rank = naive_u.shape[-1]
  assert rank == v.shape[-1] and rank == w.shape[-1]
  print(f'{key}: rank={naive_u.shape[-1]}')

# Experiment

## Floats

In [ ]:
sizes_to_be_tested = [2,3,4,5,9,10,11]
random_seed_value = 2
runs = 100

alpha_results = []
naive_results = []
strassen_results = []

for trial in sizes_to_be_tested:
    print(f"Trial number {trial}")
    size = (trial,trial,trial)
    key = f"{trial},{trial},{trial}"

    alpha_standard_factor = standard_factorizations[key]
    naive_factor = generate_naive_factorization(size)
    strassen_factor = _get_2x2x2_strassen()

    n = _get_n_from_factors(alpha_standard_factor)
    
    (full_a,full_b) = _generate_random_matrices(size, random_seed_value)
    a = block_split(full_a,n,n)
    b = block_split(full_b,n,n)

    if(trial%2 != 0):
        padded_size = int(n+1)

        offsets=[0,0]
        array=full_a
        print([slice(offsets[dim], offsets[dim] + array.shape[dim]) for dim in range(array.ndim)])
        padded_array_a = pad(full_a, (padded_size,padded_size),[0,0])
        padded_array_b = pad(full_b, (padded_size,padded_size),[0,0])

        strassen_n = padded_size/(padded_size/2)
        strassen_a = block_split(padded_array_a,strassen_n,strassen_n)
        strassen_b = block_split(padded_array_b,strassen_n,strassen_n)
    else:
        strassen_n = n/(n/2)
        strassen_a = block_split(full_a,strassen_n,strassen_n)
        strassen_b = block_split(full_b,strassen_n,strassen_n)

    alpha_standard_func = algorithm_from_factors(alpha_standard_factor)
    naive_func = algorithm_from_factors(naive_factor)
    strassen_func = algorithm_from_factors(strassen_factor)


    execution_time_alpha = timeit.timeit(
        stmt='alpha_standard_func(a, b)',
        number=runs,  
        globals=globals()
    )
    alpha_avg_time = execution_time_alpha / runs
    alpha_results.append(alpha_avg_time)
    print(f"Average execution time for Alpha over {runs} runs: {alpha_avg_time} seconds")

    execution_time_naive = timeit.timeit(
        stmt='naive_func(a, b)',
        number=runs,  
        globals=globals()
    )
    naive_avg_time = execution_time_naive / runs
    naive_results.append(naive_avg_time)
    print(f"Average execution time for Naive over {runs} runs: {naive_avg_time} seconds")

    execution_time_strassen = timeit.timeit(
        stmt='strassen_func(strassen_a, strassen_b)',
        number=runs,  
        globals=globals()
    )
    strassen_avg_time = execution_time_strassen / runs
    strassen_results.append(strassen_avg_time)
    print(f"Average execution time for Strassen over {runs} runs: {strassen_avg_time} seconds")

print("Alpha Results:", alpha_results)
print("Naive Results:", naive_results)
print("Strassen Results:", strassen_results)
results_df = pd.DataFrame({
        'Trial': sizes_to_be_tested,
        'Alpha_Avg_Time': alpha_results,
        'Naive_Avg_Time': naive_results,
        'Strassen_Avg_Time': strassen_results
    })

results_df.to_csv('/Users/kev/Documents/projects/alphatensor-analysis/results/results_float.csv', index=False)

## Ints

In [ ]:
sizes_to_be_tested = [2,3,4,5,9,10,11]
random_seed_value = 2
runs = 100

alpha_results = []
naive_results = []
strassen_results = []

for trial in sizes_to_be_tested:
    print(f"Trial number {trial}")
    size = (trial,trial,trial)
    key = f"{trial},{trial},{trial}"

    alpha_standard_factor = standard_factorizations[key]
    naive_factor = generate_naive_factorization(size)
    strassen_factor = _get_2x2x2_strassen()

    n = _get_n_from_factors(alpha_standard_factor)
    
    (full_a,full_b) = _generate_random_int_matrices(size, random_seed_value)
    a = block_split(full_a,n,n)
    b = block_split(full_b,n,n)

    if(trial%2 != 0):
        padded_size = int(n+1)

        offsets=[0,0]
        array=full_a
        print([slice(offsets[dim], offsets[dim] + array.shape[dim]) for dim in range(array.ndim)])
        padded_array_a = pad(full_a, (padded_size,padded_size),[0,0])
        padded_array_b = pad(full_b, (padded_size,padded_size),[0,0])

        strassen_n = padded_size/(padded_size/2)
        strassen_a = block_split(padded_array_a,strassen_n,strassen_n)
        strassen_b = block_split(padded_array_b,strassen_n,strassen_n)
    else:
        strassen_n = n/(n/2)
        strassen_a = block_split(full_a,strassen_n,strassen_n)
        strassen_b = block_split(full_b,strassen_n,strassen_n)

    alpha_standard_func = algorithm_from_factors(alpha_standard_factor)
    naive_func = algorithm_from_factors(naive_factor)
    strassen_func = algorithm_from_factors(strassen_factor)



    execution_time_alpha = timeit.timeit(
        stmt='alpha_standard_func(a, b)',
        number=runs,  
        globals=globals()
    )
    alpha_avg_time = execution_time_alpha / runs
    alpha_results.append(alpha_avg_time)
    print(f"Average execution time for Alpha over {runs} runs: {alpha_avg_time} seconds")

    execution_time_naive = timeit.timeit(
        stmt='naive_func(a, b)',
        number=runs,  
        globals=globals()
    )
    naive_avg_time = execution_time_naive / runs
    naive_results.append(naive_avg_time)
    print(f"Average execution time for Naive over {runs} runs: {naive_avg_time} seconds")

    execution_time_strassen = timeit.timeit(
        stmt='strassen_func(strassen_a, strassen_b)',
        number=runs,  
        globals=globals()
    )
    strassen_avg_time = execution_time_strassen / runs
    strassen_results.append(strassen_avg_time)
    print(f"Average execution time for Strassen over {runs} runs: {strassen_avg_time} seconds")

print("Alpha Results:", alpha_results)
print("Naive Results:", naive_results)
print("Strassen Results:", strassen_results)
results_df = pd.DataFrame({
        'Trial': sizes_to_be_tested,
        'Alpha_Avg_Time': alpha_results,
        'Naive_Avg_Time': naive_results,
        'Strassen_Avg_Time': strassen_results
    })

results_df.to_csv('/Users/kev/Documents/projects/alphatensor-analysis/results/results_int.csv', index=False)